In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd
data=gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)
    

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


In [5]:
##data preprocessing
import numpy as np 
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [6]:
with open('hamlet.txt','r') as file:
    text=file.read().lower()
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [8]:
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [11]:
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [21]:
x,y=input_sequences[:,:-1],input_sequences[:,-1]
print(x)
print(y)

[[   0    0    0 ...    0    0    1]
 [   0    0    0 ...    0    1  687]
 [   0    0    0 ...    1  687    4]
 ...
 [   0    0    0 ...  687    4   45]
 [   0    0    0 ...    4   45 1047]
 [   0    0    0 ...   45 1047    4]]
[ 687    4   45 ... 1047    4  193]


In [22]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y
y.shape


(25732, 4818)

In [31]:
#train our LSTM RNN
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout

model=Sequential()

model.add(Embedding(total_words,100,input_length=max_sequence_len))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))
model.build(input_shape=(None, max_sequence_len)) 
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 14, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 14, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.0305 - loss: 7.1433 - val_accuracy: 0.0295 - val_loss: 6.8059
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.0330 - loss: 6.4473 - val_accuracy: 0.0383 - val_loss: 6.9207
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.0441 - loss: 6.2987 - val_accuracy: 0.0410 - val_loss: 6.9632
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 23s 35ms/step - accuracy: 0.0502 - loss: 6.1663 - val_accuracy: 0.0406 - val_loss: 7.0037
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.0531 - loss: 6.0383 - val_accuracy: 0.0453 - val_loss: 7.0388
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 29ms/step - accuracy: 0.0618 - loss: 5.8710 - val_accuracy: 0.0472 - val_loss: 7.0743
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.0664 - loss: 5.7556 - val_accuracy: 0.0521 - val_loss: 7.1087
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - accuracy: 0.0739 - loss: 5

In [41]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list=token_list[-(max_sequence_len-1)]
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [55]:
input_text="i will take all your"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word predicted:{next_word}")

Input text:i will take all your
Next word predicted:worm


In [43]:
model.save("next_word_lstm.h5")

In [46]:
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)